# Imports

In [1]:
#imports

import matplotlib.pyplot as plt
import seaborn as sb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import numpy as np

import torchvision
import torchvision.transforms as transforms

from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm
import pandas as pd

import NoiseFactory as NF
import TransformationFactory as TF
from resnet20 import ResNetCIFAR

import test_dataset as test_ds

# 1. Resnet-20 Class & Instantiation Functions

In [2]:
def instantiate_ResNet_model():
    # specify the device for computation
    #############################################
    # your code here
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if device =='cuda':
        print("Run on GPU...")
    else:
        print("Run on CPU...")

    # Model Definition
    net = ResNetCIFAR(num_layers=20, Nbits=None) # from HW 4
    net = net.to(device)
    #############################################
    return net, device

In [3]:
def set_up_loss_function(device,
                         net,
                         INITIAL_LR: float=0.01,
                         MOMENTUM: float=0.9,
                         regularization_method: str="L2",
                         REG: float=0.0001):
    # hyperparameters, do NOT change right now
    # initial learning rate
    INITIAL_LR = INITIAL_LR

    # momentum for optimizer
    MOMENTUM = MOMENTUM

    regularization_method = regularization_method
    # L2 regularization strength
    REG = REG                  # <--- used to control both L2 and L1 regularization methods

    #############################################
    # your code here
    # create loss function
    # add L1 regularization here.
    criterion = nn.CrossEntropyLoss().to(device)

    # Add optimizer
    if regularization_method == "L2":
        optimizer = optim.SGD(net.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    else:
        optimizer = optim.SGD(net.parameters(), lr=INITIAL_LR, momentum=MOMENTUM)
    #############################################

    return criterion, optimizer

# 2. Data Transformation and Data Loader Functions

In [4]:
def get_training_transforms(intensity:int=0, noise = None, p:float=0.5):
    if intensity is None:
        intensity = 0

    n_factory = NF.NoiseFactory()
    transforms_factory = TF.TransformationFactory()

    transform_train = transforms_factory.get_transformation_composition_by_name(transformation_set=noise, intensity=intensity, p=p, train_test_validation='train')
    noise_factory_function = n_factory.get_noise_by_name(noise_type_name=noise, intensity=intensity, p=p)
    transform_val =transforms_factory.get_transformation_composition_by_name(transformation_set=None, p=p, train_test_validation='val')

    return transform_train, transform_val, noise_factory_function

In [5]:
def build_train_validation_sets(TRAIN_BATCH_SIZE: int =128,
                                VAL_BATCH_SIZE: int = 100,
                                training_noise_type: str = None,
                                training_noise_type_intensity: int = None,
                                probability_of_being_applied: float = 0.5):

    # set transformations
    transform_train, transform_val, noise_factory_function = get_training_transforms(noise=training_noise_type,
                                                                                     intensity=training_noise_type_intensity,
                                                                                     p=probability_of_being_applied)

    # a few arguments, do NOT change these
    DATA_ROOT = "./data"
    TRAIN_BATCH_SIZE = TRAIN_BATCH_SIZE
    VAL_BATCH_SIZE = VAL_BATCH_SIZE

    #############################################
    # your code here
    # construct dataset
    train_set = test_ds.CIFAR10(root='./data', train=True, download=True, transform=transform_train, imgaug_types=noise_factory_function)
    # train_set = CIFAR10(
    #     root=DATA_ROOT,
    #     mode='train',
    #     download=True,
    #     transform=transform_train,    # your code
    #     imgaug_types=noise_factory_function
    # )
    val_set = CIFAR10(
        root=DATA_ROOT,
        mode='val',
        download=True,
        transform=transform_val    # your code
    )

    # construct dataloader
    train_loader = DataLoader(
        train_set,
        batch_size=TRAIN_BATCH_SIZE ,  # your code
        shuffle=True,     # your code
        num_workers=4
    )
    val_loader = DataLoader(
        val_set,
        batch_size=VAL_BATCH_SIZE,  # your code
        shuffle=True,     # your code, shuffle to increase the random order the images are presented to prevent the model from experinceing cycles (which will make it harder to converge)
        num_workers=4
    )
    #############################################

    return train_set, val_set, train_loader, val_loader

# 3. Train/Validation function

In [6]:
def resnet_train_model(TRAIN_BATCH_SIZE: int = 128, # agnostic to transformations
                       VAL_BATCH_SIZE: int = 100,
                       training_noise_type: str = None,
                       training_noise_type_intensity: int = 0,
                       probability_of_being_applied: float = 0.5,
                       training_epochs: int = 200,
                       model_name: str = "NO_NOISE_MODEL.pt",
                       INITIAL_LR: float = 0.01,
                       MOMENTUM: float = 0.9,
                       regularization_method: str = "L2",
                       REG: float = 0.0001
                       ):

    # 2. Build/load training/validation data sets
    #    This injects noise into the training data set
    train_set, val_set, train_loader, val_loader = build_train_validation_sets(TRAIN_BATCH_SIZE=TRAIN_BATCH_SIZE,
                                                                               VAL_BATCH_SIZE=VAL_BATCH_SIZE,
                                                                               training_noise_type=training_noise_type,
                                                                               training_noise_type_intensity=training_noise_type_intensity,
                                                                               probability_of_being_applied = probability_of_being_applied)

    # 3.A. instantiate model, and put it on a GPU if a GPU is available
    net, device = instantiate_ResNet_model()

    # 3.B. set-up criterion, and optimizer
    criterion, optimizer = set_up_loss_function(device=device,
                                                net = net,
                                                INITIAL_LR=INITIAL_LR,
                                                MOMENTUM=MOMENTUM,
                                                regularization_method=regularization_method,
                                                REG=REG)

    # the folder where the trained model is saved
    CHECKPOINT_FOLDER = "./saved_models"

    #
    # Code to train the model goes here
    #      All code from a training block in HW2/HW5 would go here
    current_lr = INITIAL_LR
    best_validation_acc = 0
    print("==> Training starts!")
    print("="*50)
    vald_acc = []
    lr_reset_min = 20
    initial_lr = True
    vald_loss = []

    for i in range(0, training_epochs):
        # handle the learning rate scheduler.
        lr_reset_min -= 1
        if lr_reset_min<=0:
            if initial_lr:
                current_lr=0.1
                lr_reset_min = 40
                initial_lr = False
                for param_group in optimizer.param_groups:
                    param_group['lr'] = current_lr
                    print("Current learning rate has decayed to %f" %current_lr)
            elif abs(vald_acc[i-1] - np.mean(vald_acc[i-9:i]))*100 < 5:
                current_lr = current_lr * 0.1
                lr_reset_min = 60
                for param_group in optimizer.param_groups:
                    param_group['lr'] = current_lr
                    print("Current learning rate has decayed to %f" %current_lr)
        ######################
        # switch to train mode
        net.train()
        ######################
        print("Epoch %d:" %i)
        total_examples = 0 # this helps you compute the training accuracy
        correct_examples = 0
        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader): # this might really need to be changed over to something new
            # copy inputs to device
            inputs = inputs.to(device)
            targets = targets.to(device)

            # compute the output and loss
            outputs = net(inputs)
            # loss = criterion(outputs, targets)
            if regularization_method == "L1":
                L1_sum = 0.0
                for name, module in net.named_modules():
                    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                        L1_sum = L1_sum + torch.sum(torch.abs(module.weight))
                loss = criterion(outputs, targets) + REG * L1_sum
            else:
                loss = criterion(outputs, targets)

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            _, predicted = torch.max(outputs, 1)
            correct = predicted.eq(targets).sum()
            total_examples += targets.shape[0]
            train_loss  += loss
            correct_examples += correct.item()
            ####################################

        avg_loss = train_loss / len(train_loader)
        avg_acc = correct_examples / total_examples
        print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

        # ######################
        # # switch to val mode
        # net.eval()
        # ######################
        # total_examples = 0
        # correct_examples = 0
        #
        # val_loss = 0 # again, track the validation loss if you want
        #
        # # disable gradient during validation, which can save GPU memory
        # with torch.no_grad():
        #     for batch_idx, (inputs, targets) in enumerate(val_loader):
        #         ####################################
        #         # your code here
        #         # copy inputs to device
        #         inputs = inputs.to(device)
        #         targets = targets.to(device)
        #
        #         # compute the output and loss
        #         outputs = net(inputs)
        #         loss = criterion(outputs, targets)
        #
        #         # count the number of correctly predicted samples in the current batch
        #         _, predicted = torch.max(outputs, 1)
        #         correct = predicted.eq(targets).sum()
        #         total_examples += targets.shape[0]
        #         val_loss  += loss
        #         correct_examples += correct.item()
        #         ####################################

        # avg_loss = val_loss / len(val_loader)
        # avg_acc = correct_examples / total_examples
        # print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
        vald_acc.append(avg_acc)
        vald_loss.append(float(avg_loss))

        # Make sure to save the best model checkpoint for this noise type
        if avg_acc > best_validation_acc:
            best_validation_acc = avg_acc
            if not os.path.exists(CHECKPOINT_FOLDER):
                os.makedirs(CHECKPOINT_FOLDER)
            print("Saving ...")
            state = {'state_dict': net.state_dict(),
                     'epoch': i,
                     'lr': current_lr}
            torch.save(state, os.path.join(CHECKPOINT_FOLDER, model_name))

    #training time data
    training_time_file_name = CHECKPOINT_FOLDER + '/' + model_name + '_training_time_data.csv'
    valdation_df = list(zip(vald_loss, vald_acc))
    df = pd.DataFrame(valdation_df, columns = ['Loss', 'Accuracy'])
    df.to_csv(path_or_buf=training_time_file_name, sep=',',)

In [7]:
noise_type = ["standard"] #["motion-blur", "random-perspective", "random-resize-and-crop", "random-resize-and-crop", "gaussian-blur", "scale", "pepper", "pepper"]
intensity =  [0]#[ 1, 4, 3, 3, 1, 4, 4, 2] #5, 2 ]
frequency =  [1.0] #[0.3, 0.2, 0.9, 0.7, 0.6, 0.7, 0.1, 0.3]
model_name = ["base_model.pth"]#["DoE1_model_10.pth", "DoE1_model_11.pth", "DoE1_model_12.pth", "DoE1_model_13.pth", "DoE1_model_14.pth", "DoE1_model_15.pth", "DoE1_model_16.pth", "DoE1_model_17.pth", "DoE1_model_18.pth", "DoE1_model_19.pth" ]
for i in range (0, len(noise_type)):
    resnet_train_model(
        training_noise_type = noise_type[i],
        training_noise_type_intensity = intensity[i],
        probability_of_being_applied= frequency[i],
        model_name = model_name[i], training_epochs=200)

/home/taylor/Duke/ECE661/projects/untitled/final_project/TransformationFactory.py:140: UserWarning: TransformationsFactory not supplied with an expected transformation set name, so standard set supplied.
  warnings.warn("TransformationsFactory not supplied with an expected transformation set name, so standard set supplied.")
/home/taylor/Duke/ECE661/projects/untitled/final_project/NoiseFactory.py:48: UserWarning: NoiseFactory not supplied with an expected name, so noNoise supplied.
  warnings.warn("NoiseFactory not supplied with an expected name, so noNoise supplied.")


Files already downloaded and verified
Using downloaded and verified file: ./data/cifar10_trainval_F22.zip
Extracting ./data/cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Run on GPU...
==> Training starts!
Epoch 0:
Training loss: 1.6016, Training accuracy: 0.4097
Saving ...
Epoch 1:
Training loss: 1.1849, Training accuracy: 0.5708
Saving ...
Epoch 2:
Training loss: 0.9819, Training accuracy: 0.6483
Saving ...
Epoch 3:
Training loss: 0.8557, Training accuracy: 0.6962
Saving ...
Epoch 4:
Training loss: 0.7674, Training accuracy: 0.7282
Saving ...
Epoch 5:
Training loss: 0.7031, Training accuracy: 0.7531
Saving ...
Epoch 6:
Training loss: 0.6549, Training accuracy: 0.7706
Saving ...
Epoch 7:
Training loss: 0.6191, Training accuracy: 0.7840
Saving ...
Epoch 8:
Training loss: 0.5807, Training accuracy: 0.7972
Saving ...
Epoch 9:
Training loss: 0.5543, Training accuracy: 0.8073
Saving ...
Epoch 10:
Training loss: 0.5259, Training accuracy: 0.8168
Saving ...
Epoch 11


KeyboardInterrupt



# 4. Test Function

In [8]:
def get_test_transforms(intensity:int=0, noise = None, p:float=0.5):
    if intensity is None:
        intensity = 0

    n_factory = NF.NoiseFactory()
    transforms_factory = TF.TransformationFactory()

    transform_test = transforms_factory.get_transformation_composition_by_name(transformation_set=noise, intensity=intensity, p=p, train_test_validation='test')
    noise_factory_function = n_factory.get_noise_by_name(noise_type_name=noise, intensity=intensity, p=p)

    return transform_test, noise_factory_function

In [9]:
def get_test_data(test_noise_transformation: str = None,
                  test_noise_transformation_intensity: int = None):
    DATA_ROOT = "./data"
    transform_test, noise_factory_function = get_test_transforms(intensity=test_noise_transformation_intensity, noise = test_noise_transformation, p=1.0)
    test_set = CIFAR10(
        root=DATA_ROOT,
        mode='test',
        download=True,
        transform=transform_test,
        imgaug_types=noise_factory_function
    )

    return test_set

In [10]:
def test(net):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),

    ])

    testset = test_ds.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=1)

    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    criterion = nn.CrossEntropyLoss()

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test accuracy=%.4f" % (test_loss / (num_val_steps), val_acc))
    return val_acc

In [11]:
def test_function(model_to_test: str = 'resnet',
                  BATCH_SIZE:int = 100,
                  test_noise_transformation: str = None,
                  test_noise_transformation_intensity: int = None,
                  ):
    model_path = f'./saved_models/{model_to_test}'
    BATCH_SIZE = BATCH_SIZE

    # # inject noise into the test data
    # test_set = get_test_data(test_noise_transformation=test_noise_transformation,
    #                          test_noise_transformation_intensity=test_noise_transformation_intensity)
    #
    # # do NOT shuffle your test data loader!!!!!!!!!!!!!!!!
    # # otherwise the order of samples will be messed up
    # # and your test accuracy is likely to drop to random guessing level
    # test_loader = DataLoader(
    #     test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    #########################################################
    # use your model to generate predictions on test data
    # and save the results into variable "results"
    # "results" should be either a numpy array or a torch tensor with length of 10000

    # initialize a resnet and load trained weights
    # net, device = instantiate_ResNet_model()
    # checkpoint = torch.load(model_path) # change the path to your own checkpoint file
    # net.load_state_dict(checkpoint['state_dict'])
    # net.cuda()

    net = ResNetCIFAR()
    checkpoint = torch.load('./saved_models/base_model.pth') # change the path to your own checkpoint file
    net.load_state_dict(checkpoint['state_dict'])
    net.cuda()

    acc = test(net)


    return acc # this should be a single accuracy number

def run_testing():
    # net, device = instantiate_ResNet_model()
    list_of_model_names = ["resnetnn.pth"]
    adversarial_noises = ["scale", "salt", "gaussian-blur", "pepper", "coarse-pepper", "motion-blur", "random-resized-crop",  "random-perspective"]
    intensity_levels = [1, 2, 3, 4, 5]
    noises_dict = {}
    models_dict = {}

    #
    # This loop creates a dictionary of dictionaries that contains lists (i.e. the outer most dictionary contains entries for each model, and each of those entries is it self a dictionary that contsin entries for each adversarial noise type. Each adversarial noise type entry is a list, and those lists contain the accuracy results of the test in order from least intense to most intense)
    #
    for model in list_of_model_names:
        for noise_type in adversarial_noises:
            results = []
            for intensity_level in intensity_levels:
                result = test_function(model_to_test = model,
                                       BATCH_SIZE = 100,
                                       test_noise_transformation=noise_type,
                                       test_noise_transformation_intensity=intensity_level)
                results.append(result)
            noises_dict[noise_type] = results
        models_dict[model] = noises_dict
    print(models_dict)
    return models_dict

    # The models_dict gets fed to the heatmap plotting function

run_testing()


Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739
Files already downloaded and verified
Test Loss=0.9654, Test accuracy=0.6739

{'resnetnn.pth': {'scale': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'salt': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'gaussian-blur': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'pepper': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'coarse-pepper': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'motion-blur': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'random-resized-crop': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739],
  'random-perspective': [0.6739, 0.6739, 0.6739, 0.6739, 0.6739]}}

# 5. Plotting Devices

# 5. Plotting Devices